### Importing required libraries

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
from pandas import Timestamp

### Reading raw data files

In [2]:
train = pd.read_excel('Train_data.xlsx')
test = pd.read_excel('test_Data.xlsx')
bureau = pd.read_excel('Train_bureau.xlsx')
bureautest = pd.read_excel('test_bureau.xlsx')

Merging train and test data

In [3]:
bureau = bureau.append(bureautest)
train = train.append(test)

C:\Users\nsheokan\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Created weighted EMI (amount need to be paid per month)

In [4]:
mapping={'Half Yearly':1/6,
         'Quatrly':1/3,
         'Monthly':1,
         'BI-Monthly':2}

train['EMI_wgt'] = train['Frequency'].replace(mapping)*train.EMI

Creating top_up proxy variable for analysis

In [5]:
mapping = {'12-18 Months':1,
          '18-24 Months':1,
          '24-30 Months':1,
          '30-36 Months':1,
          '36-48 Months':1,
          ' > 48 Months':1,
          'No Top-up Service':0}

train['top_up'] = train['Top-up Month'].replace(mapping)

### Creating ratio varibles based on EMI, Income and Loan Amount

In [6]:
train['paid_rat'] = ((train['EMI_wgt']*train.Tenure)/train.DisbursalAmount)
train['emi_to_inc'] = ((train['EMI_wgt'])/train.MonthlyIncome)
train['loan_to_inc'] = ((train['DisbursalAmount'])/train.MonthlyIncome)
train['disb_to_fin'] = train.AmountFinance*0.98 > train.DisbursalAmount
train.head()

,AGE,AmountFinance,Area,AssetCost,AssetID,AuthDate,BranchID,City,DisbursalAmount,DisbursalDate,EMI,Frequency,ID,InstlmentMode,LTV,LoanStatus,ManufacturerID,MaturityDAte,MonthlyIncome,PaymentMode,SEX,State,SupplierID,Tenure,Top-up Month,ZiPCODE,EMI_wgt,top_up,paid_rat,emi_to_inc,loan_to_inc,disb_to_fin
0,49.0,275000.0,NaN,450000,4022465,2012-02-10,1,RAISEN,275000.0,2012-02-10,24000.0,Monthly,1,Arrear,61.11,Closed,1568.0,2016-01-15,35833.33,PDC_E,M,MADHYA PRADESH,21946,48,> 48 Months,464993.0,24000.000000,1.0,4.189091,0.669768,7.674419,False
1,23.0,350000.0,BHOPAL,485000,4681175,2012-03-31,333,SEHORE,350000.0,2012-03-31,10500.0,Monthly,2,Advance,70.00,Closed,1062.0,2016-02-15,666.67,PDC,M,MADHYA PRADESH,34802,47,No Top-up Service,466001.0,10500.000000,0.0,1.410000,15.749921,524.997375,False
2,39.0,519728.0,NaN,690000,25328146,2017-06-17,1,BHOPAL,519728.0,2017-06-17,38300.0,Quatrly,3,Arrear,69.77,Active,1060.0,2023-02-10,45257.00,Direct Debit,M,MADHYA PRADESH,127335,68,12-18 Months,462030.0,12766.666667,1.0,1.670361,0.282093,11.483925,False
3,24.0,400000.0,GUNA,480000,13021591,2013-11-29,125,ASHOK NAGAR,400000.0,2013-11-29,11600.0,Monthly,7,Advance,80.92,Closed,1060.0,2017-11-10,20833.33,Billed,M,MADHYA PRADESH,25094,48,> 48 Months,473335.0,11600.000000,1.0,1.392000,0.556800,19.200003,False
4,56.0,440000.0,BILASPUR,619265,3291320,2011-12-08,152,BILASPUR,440000.0,2011-12-08,15000.0,Monthly,8,Arrear,71.05,Closed,1046.0,2015-07-05,27313.67,Billed,M,CHATTISGARH,21853,44,36-48 Months,495442.0,15000.000000,1.0,1.500000,0.549176,16.109150,False


### Creating Demographics variables

In [7]:
a_cnt = train.groupby(['City'])['top_up'].count().rename('counts').reset_index()
a_cnt = a_cnt[a_cnt.counts > 50]
a_mean = train.groupby(['City'])['top_up'].mean().rename('city_mean').reset_index()
a_mean = a_mean.merge(a_cnt, how = 'inner', on = 'City')
train = train.merge(a_mean[['City', 'city_mean']], on = 'City', how = 'left')

a_cnt = train.groupby(['Area'])['top_up'].count().rename('counts').reset_index()
a_cnt = a_cnt[a_cnt.counts > 50]
a_mean = train.groupby(['Area'])['top_up'].mean().rename('Area_mean').reset_index()
a_mean = a_mean.merge(a_cnt, how = 'inner', on = 'Area')
train = train.merge(a_mean[['Area', 'Area_mean']], on = 'Area', how = 'left')

a_cnt = train.groupby(['State'])['top_up'].count().rename('counts').reset_index()
a_cnt = a_cnt[a_cnt.counts > 50]
a_mean = train.groupby(['State'])['top_up'].mean().rename('State_mean').reset_index()
a_mean = a_mean.merge(a_cnt, how = 'inner', on = 'State')
train = train.merge(a_mean[['State', 'State_mean']], on = 'State', how = 'left')

train['zip3'] = train.ZiPCODE//1000
a_cnt = train.groupby(['zip3'])['top_up'].count().rename('counts').reset_index()
a_cnt = a_cnt[a_cnt.counts > 50]
a_mean = train.groupby(['zip3'])['top_up'].mean().rename('zip3_mean').reset_index()
a_mean = a_mean.merge(a_cnt, how = 'inner', on = 'zip3')
train = train.merge(a_mean[['zip3', 'zip3_mean']], on = 'zip3', how = 'left')

train.head()

,AGE,AmountFinance,Area,AssetCost,AssetID,AuthDate,BranchID,City,DisbursalAmount,DisbursalDate,EMI,Frequency,ID,InstlmentMode,LTV,LoanStatus,ManufacturerID,MaturityDAte,MonthlyIncome,PaymentMode,SEX,State,SupplierID,Tenure,Top-up Month,ZiPCODE,EMI_wgt,top_up,paid_rat,emi_to_inc,loan_to_inc,disb_to_fin,city_mean,Area_mean,State_mean,zip3,zip3_mean
0,49.0,275000.0,NaN,450000,4022465,2012-02-10,1,RAISEN,275000.0,2012-02-10,24000.0,Monthly,1,Arrear,61.11,Closed,1568.0,2016-01-15,35833.33,PDC_E,M,MADHYA PRADESH,21946,48,> 48 Months,464993.0,24000.000000,1.0,4.189091,0.669768,7.674419,False,0.213265,NaN,0.185336,464.0,0.212393
1,23.0,350000.0,BHOPAL,485000,4681175,2012-03-31,333,SEHORE,350000.0,2012-03-31,10500.0,Monthly,2,Advance,70.00,Closed,1062.0,2016-02-15,666.67,PDC,M,MADHYA PRADESH,34802,47,No Top-up Service,466001.0,10500.000000,0.0,1.410000,15.749921,524.997375,False,0.225941,0.209646,0.185336,466.0,0.205381
2,39.0,519728.0,NaN,690000,25328146,2017-06-17,1,BHOPAL,519728.0,2017-06-17,38300.0,Quatrly,3,Arrear,69.77,Active,1060.0,2023-02-10,45257.00,Direct Debit,M,MADHYA PRADESH,127335,68,12-18 Months,462030.0,12766.666667,1.0,1.670361,0.282093,11.483925,False,0.241830,NaN,0.185336,462.0,0.179487
3,24.0,400000.0,GUNA,480000,13021591,2013-11-29,125,ASHOK NAGAR,400000.0,2013-11-29,11600.0,Monthly,7,Advance,80.92,Closed,1060.0,2017-11-10,20833.33,Billed,M,MADHYA PRADESH,25094,48,> 48 Months,473335.0,11600.000000,1.0,1.392000,0.556800,19.200003,False,0.181443,0.161787,0.185336,473.0,0.161538
4,56.0,440000.0,BILASPUR,619265,3291320,2011-12-08,152,BILASPUR,440000.0,2011-12-08,15000.0,Monthly,8,Arrear,71.05,Closed,1046.0,2015-07-05,27313.67,Billed,M,CHATTISGARH,21853,44,36-48 Months,495442.0,15000.000000,1.0,1.500000,0.549176,16.109150,False,0.109329,0.128435,0.107562,495.0,0.120482


### Creating Out of Pattern variables

In [8]:
train = train.merge(train.groupby(['State'])['MonthlyIncome'].quantile(0.1).rename('Zip_80_perc').reset_index(), on = 'State', how = 'left')
# train.head()
train['state_low_oop_income'] = train.Zip_80_perc > train.MonthlyIncome
train.drop(['Zip_80_perc'], axis = 1, inplace = True)

train = train.merge(train.groupby(['State'])['MonthlyIncome'].quantile(0.9).rename('Zip_80_perc').reset_index(), on = 'State', how = 'left')
# train.head()
train['state_high_oop_income'] = train.Zip_80_perc > train.MonthlyIncome
train.drop(['Zip_80_perc'], axis = 1, inplace = True)

train = train.merge(train.groupby(['State'])['AmountFinance'].quantile(0.1).rename('Zip_80_perc').reset_index(), on = 'State', how = 'left')
# train.head()
train['state_low_oop_loan'] = train.Zip_80_perc > train.MonthlyIncome
train.drop(['Zip_80_perc'], axis = 1, inplace = True)

train = train.merge(train.groupby(['State'])['AmountFinance'].quantile(0.9).rename('Zip_80_perc').reset_index(), on = 'State', how = 'left')
# train.head()
train['state_high_oop_loan'] = train.Zip_80_perc > train.MonthlyIncome
train.drop(['Zip_80_perc'], axis = 1, inplace = True)

train.head(4)

,AGE,AmountFinance,Area,AssetCost,AssetID,AuthDate,BranchID,City,DisbursalAmount,DisbursalDate,EMI,Frequency,ID,InstlmentMode,LTV,LoanStatus,ManufacturerID,MaturityDAte,MonthlyIncome,PaymentMode,SEX,State,SupplierID,Tenure,Top-up Month,ZiPCODE,EMI_wgt,top_up,paid_rat,emi_to_inc,loan_to_inc,disb_to_fin,city_mean,Area_mean,State_mean,zip3,zip3_mean,state_low_oop_income,state_high_oop_income,state_low_oop_loan,state_high_oop_loan
0,49.0,275000.0,NaN,450000,4022465,2012-02-10,1,RAISEN,275000.0,2012-02-10,24000.0,Monthly,1,Arrear,61.11,Closed,1568.0,2016-01-15,35833.33,PDC_E,M,MADHYA PRADESH,21946,48,> 48 Months,464993.0,24000.000000,1.0,4.189091,0.669768,7.674419,False,0.213265,NaN,0.185336,464.0,0.212393,False,True,True,True
1,23.0,350000.0,BHOPAL,485000,4681175,2012-03-31,333,SEHORE,350000.0,2012-03-31,10500.0,Monthly,2,Advance,70.00,Closed,1062.0,2016-02-15,666.67,PDC,M,MADHYA PRADESH,34802,47,No Top-up Service,466001.0,10500.000000,0.0,1.410000,15.749921,524.997375,False,0.225941,0.209646,0.185336,466.0,0.205381,True,True,True,True
2,39.0,519728.0,NaN,690000,25328146,2017-06-17,1,BHOPAL,519728.0,2017-06-17,38300.0,Quatrly,3,Arrear,69.77,Active,1060.0,2023-02-10,45257.00,Direct Debit,M,MADHYA PRADESH,127335,68,12-18 Months,462030.0,12766.666667,1.0,1.670361,0.282093,11.483925,False,0.241830,NaN,0.185336,462.0,0.179487,False,True,True,True
3,24.0,400000.0,GUNA,480000,13021591,2013-11-29,125,ASHOK NAGAR,400000.0,2013-11-29,11600.0,Monthly,7,Advance,80.92,Closed,1060.0,2017-11-10,20833.33,Billed,M,MADHYA PRADESH,25094,48,> 48 Months,473335.0,11600.000000,1.0,1.392000,0.556800,19.200003,False,0.181443,0.161787,0.185336,473.0,0.161538,False,True,True,True


## Bureau data preprocessing

In [9]:
# bureau_x = bureau.copy()

bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',0,',',00,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',1,',',10,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',2,',',20,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',3,',',30,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',4,',',40,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',5,',',50,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',6,',',60,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',7,',',70,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',8,',',80,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',9,',',90,')
bureau['DISBURSED-AMT/HIGH CREDIT'] = bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace(',','').fillna(0).astype(int)

bureau['DISBURSED-DT'] = bureau['DISBURSED-DT'].apply(lambda x: Timestamp(x))
bureau['DATE-REPORTED'] = bureau['DATE-REPORTED'].apply(lambda x: Timestamp(x))
bureau['LAST-PAYMENT-DATE'] = bureau['LAST-PAYMENT-DATE'].apply(lambda x: Timestamp(x))
# bureau['DISBURSED-DT'] = bureau['DISBURSED-DT'].apply(lambda x: Timestamp(x))


### Getting additional features from bureau for Internal trade by merging on Amount and Disbursal date

In [10]:

bureau1 = pd.merge(bureau, train[['ID', 'DisbursalDate', 'DisbursalAmount', 'Top-up Month']], on = 'ID', how = 'left')

bureau1['mon_diff'] = ((bureau1['DISBURSED-DT'] - bureau1['DisbursalDate'])/np.timedelta64(1, 'M'))
bureau1['mon_diff'] = round(bureau1.mon_diff)
bureau2 = bureau1[(bureau1['DISBURSED-AMT/HIGH CREDIT'] == bureau1.DisbursalAmount) & (bureau1['DISBURSED-DT'] == bureau1.DisbursalDate)]



train = pd.merge(train, bureau2[['ID','ACCT-TYPE', 'OWNERSHIP-IND', 'ACCOUNT-STATUS']], how = 'left', on = 'ID')
# train.head(2)

In [11]:
bureau_x = bureau1.copy()

In [12]:
bureau_x['REPORTED DATE - HIST1'] = bureau_x['REPORTED DATE - HIST'].astype(str).apply(lambda x: x.split(','))
bureau_x['CUR BAL - HIST1'] = bureau_x['CUR BAL - HIST'].astype(str).apply(lambda x: x.split(','))
bureau_x['AMT OVERDUE - HIST1'] = bureau_x['AMT OVERDUE - HIST'].astype(str).apply(lambda x: x.split(','))
bureau_x['AMT PAID - HIST1'] = bureau_x['AMT PAID - HIST'].astype(str).apply(lambda x: x.split(','))


bureau_x.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,DisbursalDate,DisbursalAmount,Top-up Month,mon_diff,REPORTED DATE - HIST1,CUR BAL - HIST1,AMT OVERDUE - HIST1,AMT PAID - HIST1
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaN,2018-02-27,NaN,37352,NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN,2012-02-10,275000.0,> 48 Months,44.0,"[20180430, 20180331, ]","[37873, 12820, ]","[37873, , ]","[, , ]"
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaN,2019-12-19,NaN,44000,"1,405/Monthly","20,797",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0,2012-02-10,275000.0,> 48 Months,73.0,"[20191231, 20191130, 20191031, 20190930, 20190...","[20797, 21988, 23174, 24341, 25504, 26648, 277...","[, , , , , , , , , , , , , , , , , , , , 1452,...","[, , , , , , , , , , , , , , , , , , , , , , ]"
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaN,NaT,NaN,145000,NaN,"1,16,087",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN,2012-02-10,275000.0,> 48 Months,91.0,"[20200131, 20191231, 20191130, 20191031, 20190...","[116087, 116087, 145000, 145000, 145000, 14500...","[0, 0, 0, 0, 0, 0, ]","[, , , , , , ]"
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21 00:00:00,NaT,NaN,300000,NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2012-02-10,275000.0,> 48 Months,20.0,"[20170930, 20170801, 20170731, 20170630, 20170...","[0, , 15925, 23754, 31494, 39147, 46713, 54194...","[0, , 1014, 1014, 1014, 1014, 1014, 1014, 1014...","[, , , , , , , , , , , , , , , , , , , , , , ,..."
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01 00:00:00,NaT,NaN,275000,NaN,0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2012-02-10,275000.0,> 48 Months,0.0,"[20160229, 20160131, 20151231, 20151130, 20151...","[0, 0, 23658, 23321, 22989, 46321, 45662, 4501...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[, , , , , , , , , , , , , , , , , , , , , , ,..."


In [13]:
print(bureau_x.shape)
# bureau_x = bureau_x[bureau_x['REPORTED DATE - HIST1'].apply(lambda x: len(x)) == bureau_x['DPD - HIST1'].apply(lambda x: len(x))]
# print(bureau_x.shape)

(624863, 33)


In [14]:
bureau4 = bureau_x[['ID', 'CONTRIBUTOR-TYPE', 'ASSET_CLASS', 'DISBURSED-DT', 'DATE-REPORTED', 'DisbursalDate', 'REPORTED DATE - HIST1', 'CUR BAL - HIST1', 'AMT OVERDUE - HIST1', 'AMT PAID - HIST1']]
bureau4.head()

bureau4['bureau_monthly_feeds'] = bureau4['REPORTED DATE - HIST1'] + bureau4['CUR BAL - HIST1'] + bureau4['AMT OVERDUE - HIST1'] + bureau4['AMT PAID - HIST1']

bureau4['monthly_snapshot'] = bureau4['bureau_monthly_feeds'].apply(lambda a: [[a[i],a[len(a)//4+i],a[2*len(a)//4+i],a[3*len(a)//4+i]] for i in range(len(a)//4)])
bureau4.head()

m_explode = bureau4[['ID', 'monthly_snapshot']].monthly_snapshot.explode().reset_index()
m_explode.head()
bureau4 = bureau4.reset_index()

bureau5 = bureau4.merge(m_explode, on = 'index', how = 'left')[['ID', 'CONTRIBUTOR-TYPE', 'ASSET_CLASS', 'DisbursalDate', 'DISBURSED-DT', 'monthly_snapshot_y']]

bureau5.head()

C:\Users\nsheokan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\nsheokan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,CONTRIBUTOR-TYPE,ASSET_CLASS,DisbursalDate,DISBURSED-DT,monthly_snapshot_y
0,1,NAB,Standard,2012-02-10,2015-10-05,"[20180430, 37873, 37873, ]"
1,1,NAB,Standard,2012-02-10,2015-10-05,"[20180331, 12820, , ]"
2,1,NAB,Standard,2012-02-10,2015-10-05,"[, , , ]"
3,1,NAB,Standard,2012-02-10,2018-03-19,"[20191231, 20797, , ]"
4,1,NAB,Standard,2012-02-10,2018-03-19,"[20191130, 21988, , ]"


In [15]:
# bureau5['monthly_snapshot_y'] = bureau5['monthly_snapshot_y'].replace('2151130', '20151130')
bureau5['raw_date'] = np.where((bureau5['monthly_snapshot_y'].apply(lambda x: x[0]).replace('',np.nan).astype(float)> 20000101) & (bureau5['monthly_snapshot_y'].apply(lambda x: x[0]).replace('',np.nan).astype(float)< 20210101), bureau5['monthly_snapshot_y'].apply(lambda x: x[0]).replace('',np.nan).astype(float), np.nan)
bureau5 = bureau5[-bureau5.raw_date.isnull()]
bureau5['report_date'] = bureau5['raw_date'].apply(lambda x: Timestamp(str(int(x))))
bureau5['current_bal'] = bureau5['monthly_snapshot_y'].apply(lambda x: (x[1]))
bureau5['overdue_bal'] = bureau5['monthly_snapshot_y'].apply(lambda x: (x[2]))
bureau5['paid_bal'] = bureau5['monthly_snapshot_y'].apply(lambda x: (x[3]))
bureau5.head()

,ID,CONTRIBUTOR-TYPE,ASSET_CLASS,DisbursalDate,DISBURSED-DT,monthly_snapshot_y,raw_date,report_date,current_bal,overdue_bal,paid_bal
0,1,NAB,Standard,2012-02-10,2015-10-05,"[20180430, 37873, 37873, ]",20180430.0,2018-04-30,37873,37873,
1,1,NAB,Standard,2012-02-10,2015-10-05,"[20180331, 12820, , ]",20180331.0,2018-03-31,12820,,
3,1,NAB,Standard,2012-02-10,2018-03-19,"[20191231, 20797, , ]",20191231.0,2019-12-31,20797,,
4,1,NAB,Standard,2012-02-10,2018-03-19,"[20191130, 21988, , ]",20191130.0,2019-11-30,21988,,
5,1,NAB,Standard,2012-02-10,2018-03-19,"[20191031, 23174, , ]",20191031.0,2019-10-31,23174,,


In [16]:
# bureau1['mon_diff'] = ((bureau1['DISBURSED-DT'] - bureau1['DisbursalDate'])/np.timedelta64(1, 'M'))
bureau5['mon_diff'] = (bureau5['report_date'] - bureau5['DisbursalDate'])/np.timedelta64(1, 'M')
bureau5.head()

,ID,CONTRIBUTOR-TYPE,ASSET_CLASS,DisbursalDate,DISBURSED-DT,monthly_snapshot_y,raw_date,report_date,current_bal,overdue_bal,paid_bal,mon_diff
0,1,NAB,Standard,2012-02-10,2015-10-05,"[20180430, 37873, 37873, ]",20180430.0,2018-04-30,37873,37873,,74.613442
1,1,NAB,Standard,2012-02-10,2015-10-05,"[20180331, 12820, , ]",20180331.0,2018-03-31,12820,,,73.627795
3,1,NAB,Standard,2012-02-10,2018-03-19,"[20191231, 20797, , ]",20191231.0,2019-12-31,20797,,,94.654921
4,1,NAB,Standard,2012-02-10,2018-03-19,"[20191130, 21988, , ]",20191130.0,2019-11-30,21988,,,93.636420
5,1,NAB,Standard,2012-02-10,2018-03-19,"[20191031, 23174, , ]",20191031.0,2019-10-31,23174,,,92.650773


In [17]:
bureau5['current_bal'] = bureau5.current_bal.replace('',0).astype(int)
bureau5['overdue_bal'] = bureau5.overdue_bal.replace('',0).astype(int)
bureau5['paid_bal'] = bureau5.overdue_bal.replace('',0).astype(int)
bureau5.head()

,ID,CONTRIBUTOR-TYPE,ASSET_CLASS,DisbursalDate,DISBURSED-DT,monthly_snapshot_y,raw_date,report_date,current_bal,overdue_bal,paid_bal,mon_diff
0,1,NAB,Standard,2012-02-10,2015-10-05,"[20180430, 37873, 37873, ]",20180430.0,2018-04-30,37873,37873,37873,74.613442
1,1,NAB,Standard,2012-02-10,2015-10-05,"[20180331, 12820, , ]",20180331.0,2018-03-31,12820,0,0,73.627795
3,1,NAB,Standard,2012-02-10,2018-03-19,"[20191231, 20797, , ]",20191231.0,2019-12-31,20797,0,0,94.654921
4,1,NAB,Standard,2012-02-10,2018-03-19,"[20191130, 21988, , ]",20191130.0,2019-11-30,21988,0,0,93.636420
5,1,NAB,Standard,2012-02-10,2018-03-19,"[20191031, 23174, , ]",20191031.0,2019-10-31,23174,0,0,92.650773


In [18]:
bureau_x = bureau5.copy()

In [19]:
bureau5['dpd_status'] = bureau5.overdue_bal>0
bureau5['dpd_status'] = bureau5.dpd_status.astype(int)
# bureau5.overdue_bal>0
bureau5.head()

,ID,CONTRIBUTOR-TYPE,ASSET_CLASS,DisbursalDate,DISBURSED-DT,monthly_snapshot_y,raw_date,report_date,current_bal,overdue_bal,paid_bal,mon_diff,dpd_status
0,1,NAB,Standard,2012-02-10,2015-10-05,"[20180430, 37873, 37873, ]",20180430.0,2018-04-30,37873,37873,37873,74.613442,1
1,1,NAB,Standard,2012-02-10,2015-10-05,"[20180331, 12820, , ]",20180331.0,2018-03-31,12820,0,0,73.627795,0
3,1,NAB,Standard,2012-02-10,2018-03-19,"[20191231, 20797, , ]",20191231.0,2019-12-31,20797,0,0,94.654921,0
4,1,NAB,Standard,2012-02-10,2018-03-19,"[20191130, 21988, , ]",20191130.0,2019-11-30,21988,0,0,93.636420,0
5,1,NAB,Standard,2012-02-10,2018-03-19,"[20191031, 23174, , ]",20191031.0,2019-10-31,23174,0,0,92.650773,0


In [20]:
# mapping = {True:5,
#           False:1}
# # train['Top-up Month'].replace(mapping)
# bureau5['dpd_status'] = bureau5['dpd_status'].replace(mapping)

In [21]:
aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 12) & (bureau5['mon_diff'] <18)].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_12_18m_crnt_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 12) & (bureau5['mon_diff'] <18)].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_12_18m_ovdue_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 12) & (bureau5['mon_diff'] <18))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_12_18m_crnt_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 12) & (bureau5['mon_diff'] <18))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_12_18m_ovdue_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

In [22]:
aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 24) & (bureau5['mon_diff'] <30)].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_24_30m_crnt_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 24) & (bureau5['mon_diff'] <30)].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_24_30m_ovdue_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 24) & (bureau5['mon_diff'] <30))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_24_30m_crnt_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 24) & (bureau5['mon_diff'] <30))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_24_30m_ovdue_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')


In [23]:
aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 30) & (bureau5['mon_diff'] <36)].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_30_36m_crnt_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 30) & (bureau5['mon_diff'] <36)].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_30_36m_ovdue_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 30) & (bureau5['mon_diff'] <36))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_30_36m_crnt_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 30) & (bureau5['mon_diff'] <36))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_30_36m_ovdue_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')


In [24]:
aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 36) & (bureau5['mon_diff'] <48)].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_36_48m_crnt_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 36) & (bureau5['mon_diff'] <48)].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_36_48m_ovdue_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 36) & (bureau5['mon_diff'] <48))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_36_48m_crnt_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 36) & (bureau5['mon_diff'] <48))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_36_48m_ovdue_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')


In [25]:
aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 48) & (bureau5['mon_diff'] <60)].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_48_60m_crnt_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 48) & (bureau5['mon_diff'] <60)].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_48_60m_ovdue_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 48) & (bureau5['mon_diff'] <60))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_48_60m_crnt_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 48) & (bureau5['mon_diff'] <60))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_48_60m_ovdue_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')


In [26]:
aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 60) & (bureau5['mon_diff'] <600)].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_60_600m_crnt_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[(bureau5['mon_diff'] >= 60) & (bureau5['mon_diff'] <600)].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_60_600m_ovdue_all_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 60) & (bureau5['mon_diff'] <600))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['current_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['current_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_60_600m_crnt_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')

aa = (pd.DataFrame(bureau5[((bureau5['mon_diff'] >= 60) & (bureau5['mon_diff'] <600))&(bureau5['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = ['ID', 'DISBURSED-DT'])['overdue_bal'].max()).reset_index())
aaa = aa.groupby(by = ['ID'])['overdue_bal'].sum().reset_index()
aaa.columns = ['ID', 'within_60_600m_ovdue_nbf_bal']
train = train.merge(aaa, on = 'ID', how = 'left')


In [27]:
train['Disb_year'] = train.DisbursalDate.apply(lambda x: str(x)[:4])
train['Mat_year'] = train['MaturityDAte'].apply(lambda x: str(x)[:4])
# train.head()

In [28]:
x = pd.DataFrame(bureau1.ID.value_counts()).reset_index()
x.columns = ['ID', 'total_trades']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')
# train.head()

In [29]:
x = pd.DataFrame(bureau1[(bureau1.mon_diff > 6)].ID.value_counts()).reset_index()
x.columns = ['ID', 'total_later_trades']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')
# train.head()

In [30]:
x = pd.DataFrame(bureau1[bureau1['CONTRIBUTOR-TYPE'] == 'NBF'].ID.value_counts()).reset_index()
x.columns = ['ID', 'total_nbf_trades']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')
# train.head(2)

In [31]:
x = pd.DataFrame(bureau1[(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')&(bureau1.mon_diff > 6)].ID.value_counts()).reset_index()
x.columns = ['ID', 'total_nbf_later_trades']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')
# train.head(2)

In [32]:
x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= -12.0) & (bureau1['mon_diff'] <0.0)].ID.value_counts()).reset_index()
x.columns = ['ID', 'prev_1y_all_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= -12.0) & (bureau1['mon_diff'] <0.0)&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].ID.value_counts()).reset_index()
x.columns = ['ID', 'prev_1y_nfb_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= -12.0) & (bureau1['mon_diff'] <0.0)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'prev_1y_all_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= -12.0) & (bureau1['mon_diff'] <0.0))&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'prev_1y_nfb_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

# train.head(10)

In [33]:
x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_1y_all_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12)&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_1y_nfb_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_1y_all_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12))&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_1y_nfb_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

In [34]:
x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_12_18m_all_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18)&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_12_18m_nfb_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_12_18m_all_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18))&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_12_18m_nfb_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')


In [35]:
x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_18_24m_all_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24)&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_18_24m_nfb_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_18_24m_all_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24))&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_18_24m_nfb_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')


In [36]:
x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_24_30m_all_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30)&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_24_30m_nfb_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_24_30m_all_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30))&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_24_30m_nfb_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')


In [37]:
x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_30_36m_all_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36)&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_30_36m_nfb_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_30_36m_all_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36))&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_30_36m_nfb_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')


In [38]:
x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_36_48m_all_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48)&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_36_48m_nfb_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_36_48m_all_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48))&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_36_48m_nfb_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')


In [39]:
x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_48_60m_all_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60)&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_48_60m_nfb_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_48_60m_all_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60))&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_48_60m_nfb_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')


In [40]:
x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_60_600m_all_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600)&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_60_600m_nfb_t_cnt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_60_600m_all_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600))&(bureau1['CONTRIBUTOR-TYPE'] == 'NBF')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_60_600m_nfb_t_amt']
x[x.ID == 1]
train = train.merge(x, on = 'ID', how = 'left')


In [41]:
train['prev_1y_nfb_t_amt_rat'] = train['prev_1y_nfb_t_amt']/train['DisbursalAmount']
train['prev_1y_all_t_amt_rat'] = train['prev_1y_all_t_amt']/train['DisbursalAmount']

train['within_1y_all_t_amt_rat'] = train['within_1y_all_t_amt']/train['DisbursalAmount']
train['within_1y_nfb_t_amt_rat'] = train['within_1y_nfb_t_amt']/train['DisbursalAmount']

train['within_18_24m_nfb_t_amt_rat'] = train['within_18_24m_nfb_t_amt']/train['DisbursalAmount']
train['within_18_24m_all_t_amt_rat'] = train['within_18_24m_all_t_amt']/train['DisbursalAmount']

train['within_24_30m_nfb_t_amt_rat'] = train['within_24_30m_nfb_t_amt']/train['DisbursalAmount']
train['within_24_30m_all_t_amt_rat'] = train['within_24_30m_all_t_amt']/train['DisbursalAmount']

train['within_30_36m_nfb_t_amt_rat'] = train['within_30_36m_nfb_t_amt']/train['DisbursalAmount']
train['within_30_36m_all_t_amt_rat'] = train['within_30_36m_all_t_amt']/train['DisbursalAmount']

train['within_36_48m_nfb_t_amt_rat'] = train['within_36_48m_nfb_t_amt']/train['DisbursalAmount']
train['within_36_48m_all_t_amt_rat'] = train['within_36_48m_all_t_amt']/train['DisbursalAmount']

train['within_48_60m_nfb_t_amt_rat'] = train['within_48_60m_nfb_t_amt']/train['DisbursalAmount']
train['within_48_60m_all_t_amt_rat'] = train['within_48_60m_all_t_amt']/train['DisbursalAmount']


train['within_60_600m_nfb_t_amt_rat'] = train['within_60_600m_nfb_t_amt']/train['DisbursalAmount']
train['within_60_600m_all_t_amt_rat'] = train['within_60_600m_all_t_amt']/train['DisbursalAmount']


train['within_12_18m_nfb_t_amt_rat'] = train['within_12_18m_nfb_t_amt']/train['DisbursalAmount']
train['within_12_18m_all_t_amt_rat'] = train['within_12_18m_all_t_amt']/train['DisbursalAmount']

In [42]:
x = pd.DataFrame(bureau1[bureau1['SELF-INDICATOR'] == True].ID.value_counts()).reset_index()
x.columns = ['ID', 'all_self_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[bureau1['SELF-INDICATOR'] == True].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'all_self_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12)&(bureau1['SELF-INDICATOR'] == True)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_1y_self_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12))&(bureau1['SELF-INDICATOR'] == True)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_1y_self_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18)&(bureau1['SELF-INDICATOR'] == True)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_12_18m_self_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18))&(bureau1['SELF-INDICATOR'] == True)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_12_18m_self_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24)&(bureau1['SELF-INDICATOR'] == True)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_18_24m_self_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24))&(bureau1['SELF-INDICATOR'] == True)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_18_24m_self_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30)&(bureau1['SELF-INDICATOR'] == True)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_24_30m_self_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30))&(bureau1['SELF-INDICATOR'] == True)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_24_30m_self_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36)&(bureau1['SELF-INDICATOR'] == True)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_30_36m_self_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36))&(bureau1['SELF-INDICATOR'] == True)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_30_36m_self_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48)&(bureau1['SELF-INDICATOR'] == True)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_36_48m_self_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48))&(bureau1['SELF-INDICATOR'] == True)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_36_48m_self_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60)&(bureau1['SELF-INDICATOR'] == True)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_48_60m_self_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60))&(bureau1['SELF-INDICATOR'] == True)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_48_60m_self_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600)&(bureau1['SELF-INDICATOR'] == True)].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_60_600m_self_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600))&(bureau1['SELF-INDICATOR'] == True)].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_60_600m_self_t_amt']
train = train.merge(x, on = 'ID', how = 'left')
# train.head()

In [43]:
x = pd.DataFrame(bureau1[bureau1['ACCT-TYPE'] == 'Tractor Loan'].ID.value_counts()).reset_index()
x.columns = ['ID', 'all_tloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[bureau1['ACCT-TYPE'] == 'Tractor Loan'].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'all_tloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12)&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_1y_tloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12))&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_1y_tloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18)&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_12_18m_tloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18))&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_12_18m_tloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24)&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_18_24m_tloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24))&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_18_24m_tloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30)&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_24_30m_tloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30))&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_24_30m_tloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36)&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_30_36m_tloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36))&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_30_36m_tloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48)&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_36_48m_tloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48))&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_36_48m_tloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60)&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_48_60m_tloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60))&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_48_60m_tloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600)&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_60_600m_tloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600))&(bureau1['ACCT-TYPE'] == 'Tractor Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_60_600m_tloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')
# train.head()

In [44]:
x = pd.DataFrame(bureau1[bureau1['ACCT-TYPE'] == 'Gold Loan'].ID.value_counts()).reset_index()
x.columns = ['ID', 'all_gloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[bureau1['ACCT-TYPE'] == 'Gold Loan'].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'all_gloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12)&(bureau1['ACCT-TYPE'] == 'Gold Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_1y_gloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12))&(bureau1['ACCT-TYPE'] == 'Gold Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_1y_gloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18)&(bureau1['ACCT-TYPE'] == 'Gold Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_12_18m_gloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18))&(bureau1['ACCT-TYPE'] == 'Gold Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_12_18m_gloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24)&(bureau1['ACCT-TYPE'] == 'Gold Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_18_24m_gloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24))&(bureau1['ACCT-TYPE'] == 'Gold Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_18_24m_gloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30)&(bureau1['ACCT-TYPE'] == 'Gold Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_24_30m_gloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30))&(bureau1['ACCT-TYPE'] == 'Gold Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_24_30m_gloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36)&(bureau1['ACCT-TYPE'] == 'Gold Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_30_36m_gloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36))&(bureau1['ACCT-TYPE'] == 'Gold Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_30_36m_gloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48)&(bureau1['ACCT-TYPE'] == 'Gold Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_36_48m_gloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48))&(bureau1['ACCT-TYPE'] == 'Gold Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_36_48m_gloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60)&(bureau1['ACCT-TYPE'] == 'Gold Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_48_60m_gloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60))&(bureau1['ACCT-TYPE'] == 'Gold Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_48_60m_gloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600)&(bureau1['ACCT-TYPE'] == 'Gold Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_60_600m_gloan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600))&(bureau1['ACCT-TYPE'] == 'Gold Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_60_600m_gloan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')
# train.head()

In [45]:
x = pd.DataFrame(bureau1[bureau1['ACCT-TYPE'] == 'Personal Loan'].ID.value_counts()).reset_index()
x.columns = ['ID', 'all_ploan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[bureau1['ACCT-TYPE'] == 'Personal Loan'].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'all_ploan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12)&(bureau1['ACCT-TYPE'] == 'Personal Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_1y_ploan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 0) & (bureau1['mon_diff'] <12))&(bureau1['ACCT-TYPE'] == 'Personal Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_1y_ploan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18)&(bureau1['ACCT-TYPE'] == 'Personal Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_12_18m_ploan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 12) & (bureau1['mon_diff'] <18))&(bureau1['ACCT-TYPE'] == 'Personal Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_12_18m_ploan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24)&(bureau1['ACCT-TYPE'] == 'Personal Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_18_24m_ploan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 18) & (bureau1['mon_diff'] <24))&(bureau1['ACCT-TYPE'] == 'Personal Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_18_24m_ploan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30)&(bureau1['ACCT-TYPE'] == 'Personal Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_24_30m_ploan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 24) & (bureau1['mon_diff'] <30))&(bureau1['ACCT-TYPE'] == 'Personal Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_24_30m_ploan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')

x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36)&(bureau1['ACCT-TYPE'] == 'Personal Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_30_36m_ploan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 30) & (bureau1['mon_diff'] <36))&(bureau1['ACCT-TYPE'] == 'Personal Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_30_36m_ploan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48)&(bureau1['ACCT-TYPE'] == 'Personal Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_36_48m_ploan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 36) & (bureau1['mon_diff'] <48))&(bureau1['ACCT-TYPE'] == 'Personal Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_36_48m_ploan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60)&(bureau1['ACCT-TYPE'] == 'Personal Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_48_60m_ploan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 48) & (bureau1['mon_diff'] <60))&(bureau1['ACCT-TYPE'] == 'Personal Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_48_60m_ploan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[(bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600)&(bureau1['ACCT-TYPE'] == 'Personal Loan')].ID.value_counts()).reset_index()
x.columns = ['ID', 'within_60_600m_ploan_t_cnt']
train = train.merge(x, on = 'ID', how = 'left')


x = pd.DataFrame(bureau1[((bureau1['mon_diff'] >= 60) & (bureau1['mon_diff'] <600))&(bureau1['ACCT-TYPE'] == 'Personal Loan')].groupby(by = 'ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).reset_index()
x.columns = ['ID', 'within_60_600m_ploan_t_amt']
train = train.merge(x, on = 'ID', how = 'left')
# train.head()

In [46]:
train.shape

(144405, 200)

In [47]:

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
col=[i for i in train.columns if train[i].dtypes==object]
for i in col:
    if i != 'Top-up Month':
        train[i]= label_encoder.fit_transform(train[i].fillna('AAA'))
# train.head()

In [48]:
col=['ZiPCODE', 'ManufacturerID', 'SupplierID']
for i in col:
    train[i]= label_encoder.fit_transform(train[i])
# train.head()

In [49]:
train_final = train[-train['Top-up Month'].isnull()]
train_final.shape

(129588, 200)

In [50]:
test_final = train[train['Top-up Month'].isnull()]
test_final.shape

(14817, 200)

In [51]:
to_drop = ['ID', 'AmountFinance', 'DisbursalDate', 
          'MaturityDAte', 'AuthDate', 'AssetID', 'BranchID', 'City', 'Top-up Month', 'top_up']
# to_drop = ['ID', 'AmountFinance', 'DisbursalDate', 
#           'MaturityDAte', 'AuthDate', 'AssetID', 'Top-up Month']

X = train_final.drop(to_drop, axis=1)

In [52]:
mapping = {'12-18 Months':1,
          '18-24 Months':2,
          '24-30 Months':3,
          '30-36 Months':4,
          '36-48 Months':5,
          ' > 48 Months':6,
          'No Top-up Service':9}
train['Top-up Month'].replace(mapping)
y = train_final['Top-up Month'].replace(mapping)
# y = train['Top-up Month']

In [53]:
#Splitting of data into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [54]:

# X_test = test1.drop(['ID','Top-up Month'], axis=1)
# test_id = test1.ID

# del train, test
import xgboost as xgb
clf = xgb.XGBClassifier(
        n_estimators=20,
        max_depth=9,
        learning_rate=0.071,
        subsample=0.75,
        colsample_bytree=0.9,
#     reg_lambda= 0.001
    )
clf.fit(X_train, y_train, eval_set = [(X_train, y_train)], verbose=True)

# y_preds = clf.predict(X_test)

[0]	validation_0-merror:0.08105
[1]	validation_0-merror:0.07905
[2]	validation_0-merror:0.07804
[3]	validation_0-merror:0.07757
[4]	validation_0-merror:0.07710
[5]	validation_0-merror:0.07664
[6]	validation_0-merror:0.07610
[7]	validation_0-merror:0.07587
[8]	validation_0-merror:0.07537
[9]	validation_0-merror:0.07487
[10]	validation_0-merror:0.07468
[11]	validation_0-merror:0.07449
[12]	validation_0-merror:0.07421
[13]	validation_0-merror:0.07397
[14]	validation_0-merror:0.07362
[15]	validation_0-merror:0.07323
[16]	validation_0-merror:0.07298
[17]	validation_0-merror:0.07279
[18]	validation_0-merror:0.07259
[19]	validation_0-merror:0.07224


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.071, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.75,
              tree_method=None, validate_parameters=False, verbosity=None)

In [55]:
y_preds = clf.predict(X_test)
pd.DataFrame(y_preds).iloc[:,0].value_counts()

9    23043
6     1172
5      427
3      401
2      364
4      346
1      165
Name: 0, dtype: int64

In [56]:
# added oop, ftg, 12 changed to 12
# y_preds = clf.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_preds, labels=None, pos_label=1, average='macro')

0.7028610237198054

In [57]:
import xgboost as xgb
clf = xgb.XGBClassifier(
        n_estimators=50,
        max_depth=9,
        learning_rate=0.071,
        subsample=0.75,
        colsample_bytree=0.9,
#     reg_lambda= 0.001
    )
clf.fit(X, y, eval_set = [(X, y)], verbose=True)

[0]	validation_0-merror:0.08320
[1]	validation_0-merror:0.08002
[2]	validation_0-merror:0.07835
[3]	validation_0-merror:0.07769
[4]	validation_0-merror:0.07716
[5]	validation_0-merror:0.07672
[6]	validation_0-merror:0.07644
[7]	validation_0-merror:0.07605
[8]	validation_0-merror:0.07585
[9]	validation_0-merror:0.07552
[10]	validation_0-merror:0.07534
[11]	validation_0-merror:0.07522
[12]	validation_0-merror:0.07498
[13]	validation_0-merror:0.07475
[14]	validation_0-merror:0.07456
[15]	validation_0-merror:0.07431
[16]	validation_0-merror:0.07408
[17]	validation_0-merror:0.07376
[18]	validation_0-merror:0.07366
[19]	validation_0-merror:0.07356
[20]	validation_0-merror:0.07319
[21]	validation_0-merror:0.07297
[22]	validation_0-merror:0.07261
[23]	validation_0-merror:0.07242
[24]	validation_0-merror:0.07213
[25]	validation_0-merror:0.07187
[26]	validation_0-merror:0.07162
[27]	validation_0-merror:0.07146
[28]	validation_0-merror:0.07136
[29]	validation_0-merror:0.07108
[30]	validation_0-me

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.071, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.75,
              tree_method=None, validate_parameters=False, verbosity=None)

In [58]:
to_drop = ['ID', 'AmountFinance', 'DisbursalDate', 
          'MaturityDAte', 'AuthDate', 'AssetID', 'BranchID', 'City', 'Top-up Month', 'top_up']

X_test = test_final.drop(to_drop, axis=1)

In [59]:
y_preds = clf.predict(X_test)

In [60]:
submission = test_final[['ID', 'AmountFinance']]
submission['Top-up Month'] = y_preds
# sample['Top-up Month'] = ls
rev_mapping = {1:'12-18 Months',
          2:'18-24 Months',
          3:'24-30 Months',
          4:'30-36 Months',
          5:'36-48 Months',
          6:' > 48 Months',
          9:'No Top-up Service'}
submission['Top-up Month'] = submission['Top-up Month'].replace(rev_mapping)
submission = submission.drop_duplicates()
submission.head()

C:\Users\nsheokan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\nsheokan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,ID,AmountFinance,Top-up Month
129588,4,365000.0,No Top-up Service
129589,5,285000.0,> 48 Months
129590,6,400000.0,No Top-up Service
129591,25,500000.0,No Top-up Service
129592,119,400000.0,No Top-up Service


In [61]:
sample = pd.read_csv('sample_submission_ejm25Dc.csv')
sample.head()

,ID,Top-up Month
0,4,36-48 Months
1,5,18-24 Months
2,6,30-36 Months
3,25,30-36 Months
4,119,30-36 Months


In [62]:
sample = sample[['ID']].merge(submission[['ID', 'Top-up Month']], on = 'ID', how = 'inner')
print(sample.shape)
sample.head()

(14745, 2)


,ID,Top-up Month
0,4,No Top-up Service
1,5,> 48 Months
2,6,No Top-up Service
3,25,No Top-up Service
4,119,No Top-up Service


In [63]:
sample.to_csv('Final_submission.csv', index = None)

In [64]:
sample.iloc[:,1].value_counts()

No Top-up Service    13718
 > 48 Months           486
36-48 Months           194
18-24 Months            97
12-18 Months            84
24-30 Months            83
30-36 Months            83
Name: Top-up Month, dtype: int64